

# AMLS Assignment Draft
## Task A_3: CNN on BreastMNIST Dataset Validating Learning Rate

Explore range of Learning Rates for CNN based model on the BreastMNIST dataset.

## Import libraries
The required libraries for this notebook include numpy, matplotlib, tensorflow, medmnist and also a custom built dataload and utility library.

In [2]:
## import libraries
## first enable autoreload during development so latest (new) version local code library is reloaded on execution 
## can be commented out when local code development not happening to avoid overhead
%load_ext autoreload
%autoreload 2
    
import io
import time                          ## to enable delays between file saves
import numpy as np                 
import matplotlib.pyplot as plt      ## for graphing
import pandas as pd                  ## to enable data structure changes
## import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, Hinge

## removed MedMNIST as only used in AMLS common library

## local code library
import AMLS_common as ac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set base parameters
Including hyper parameters and data set specifics

In [3]:
## set up lists and parameters
test_list     = []
run_list      = []
## use these lists of values to grid test hyper parameter sensitivity                
epochs_list   = [30,40]                         ## set of epochs to run for
filter_list   = [16,32,64]                      ## main filter sizes to use
bs_list       = [128]                           ## dataset batch size
lr_list       = [0.01, 0.001, 0.0001]           ## learning rates
ly_list       = [2,3,4]                         ## number of covolution layers
dr_list       = [0.1,0.2,0.3,0.4]                       ## selected dropout rates
loss_list     = ['binary_crossentropy','hinge'] ## loss functions to use
optimise_list = ['Adam','SGD','RMSprop']        ## optimisation functions
for ep in epochs_list:
    for bs in bs_list:
        for lr in lr_list:
            for fi in filter_list:
                for ly in ly_list:
                    for dr in dr_list:
                        for ls in loss_list:
                            for op in optimise_list:
                                parameter = ac.HyperParameters(learning_rate=lr, 
                                                               batch_size=bs, 
                                                               num_epochs=ep, 
                                                               filter=fi,
                                                               layers=ly,
                                                               dropout_rate=dr,
                                                               optimise=op,
                                                               loss=ls)          
                                test_list.append([parameter])
## reshape parameters into a test grid that can be read using for loop
test_grid = [hp for sublist in test_list for hp in sublist]


In [4]:
## control and environment (e.g. verbose) parameters
filebase   = "metrics/"          # the folder to store the metrics and summary files generated for each run
verbose    = 0                   # if value equals 1 then print additional process information in steps below

## Load and preprocess the BreastMNIST Data
We load the dataset.

In [5]:
## Loading the data file using custom MedMINST loader
data_flag  = 'breastmnist'       ## defines which dataset to load
result_set = ac.medMNIST_load(data_flag,parameter.batch_size)

## check that the loader returned data correctly and then split out
if result_set != []:
    train_dataset = result_set[0]
    test_dataset  = result_set[1]
    val_dataset   = result_set[2]

if verbose == 1:
    print("\nSummary metrics for validation dataset")
    print("type:",type(val_dataset))
    print("length:",len(val_dataset))
    print("shape:",val_dataset)

Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz


## Fit the model

Using each Hyperparameter in turn from a superset

In [6]:
## Create instances of the dataclass from the list
for item in test_grid:
    ## Define the model which is then run for all learning rates in set
    print("Run with",item)
    print(item.num_epochs)
    
    ## initialise tqdm callback
    tqdm_callback = ac.TqdmEpochProgress(total_epochs=item.num_epochs)
    
    ## Simple CNN model to support Learning Rate analysis
    ## added desired number of layers
    if item.layers == 2:
        model = Sequential([
            Conv2D(item.filter, (3, 3), activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.filter*2, (3, 3), activation=item.default_activation),                         ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Conv2D(item.filter, (3, 3), activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Flatten(),                                                                                 ## Flatten
            Dropout(item.dropout_rate),                                                                ## Drop out to reduce possible over fitting
            Dense(item.filter*4, activation=item.default_activation),                                  ## Fully connected layer
            Dense(1, activation='sigmoid')                                                             ## Output layer for binary classification  
        ])
    if item.layers == 3:
        model = Sequential([
            Conv2D(item.filter, (3, 3), activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.filter*2, (3, 3), activation=item.default_activation),                         ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Conv2D(item.filter, (3, 3), activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Conv2D(item.filter, (3, 3), activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Flatten(),                                                                                 ## Flatten
            Dropout(item.dropout_rate),                                                                ## Drop out to reduce possible over fitting
            Dense(item.filter*4, activation=item.default_activation),                                  ## Fully connected layer
            Dense(1, activation='sigmoid')                                                             ## Output layer for binary classification  
        ])
    if item.layers == 4:
        model = Sequential([
            Conv2D(item.filter, (3, 3), activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.filter*2, (3, 3), activation=item.default_activation),                         ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Conv2D(item.filter, (3, 3), activation=item.default_activation),                           ## Another convolution layer
            Conv2D(item.filter*2, (3, 3), activation=item.default_activation),                         ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Conv2D(item.filter, (3, 3), activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2)),                                                                      ## Reduce the feature maps
            Flatten(),                                                                                 ## Flatten
            Dropout(item.dropout_rate),                                                                ## Drop out to reduce possible over fitting
            Dense(item.filter*4, activation=item.default_activation),                                  ## Fully connected layer
            Dense(1, activation='sigmoid')                                                             ## Output layer for binary classification  
        ])    
        
    if verbose == 1:
        print(model.summary())
        
    ## Redirect the summary output to a string
    summary_string  = io.StringIO()
    model.summary(print_fn=lambda x: summary_string.write(x + "\n"))
    summary_content = summary_string.getvalue()
    summary_string.close()

    ## Compile the model
    model.compile(optimizer=item.optimise,                                                   
                  loss=item.loss,
                  metrics='acc')

    ## Fit the model
    history = model.fit(val_dataset, 
                        epochs=item.num_epochs, 
                        batch_size=item.batch_size, 
                        verbose=0)
## suppressed callbacks for now
    ##                        callbacks = [tqdm_callback])
    
    ## Save results to files
    run_list.append(ac.hyper_process(history,summary_content,item))

print("Hyperparameter test run complete")

Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='Adam', loss='binary_crossentropy', filter=16, filter2=16, dropout_rate=0.1, layers=2, default_activation='relu')
30
Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='SGD', loss='binary_crossentropy', filter=16, filter2=16, dropout_rate=0.1, layers=2, default_activation='relu')
30
Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='RMSprop', loss='binary_crossentropy', filter=16, filter2=16, dropout_rate=0.1, layers=2, default_activation='relu')
30
Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='Adam', loss='hinge', filter=16, filter2=16, dropout_rate=0.1, layers=2, default_activation='relu')
30
Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='SGD', loss='hinge', filter=16, filter2=16, dropout_rate=0.1, layers=2, default_activation='relu')
30
Run with HyperPar

In [7]:
best_run,run_df = ac.analyse_run(run_list)
print("\nRun satisfying both smallest min_loss and largest max_acc:")
print(best_run)

feature_importance,coef = ac.analyse_hyperparameters(run_df)
print("\nImpact of Hyperparameters on Accuracy (from Linear Regression):")
print(coef)
print("\nHyperparameter Importance for Accuracy (from Random Forest):")
print(feature_importance)


Run satisfying both smallest min_loss and largest max_acc:
     metrics_file summary_file  min_loss   max_acc  last_loss  last_acc  \
1236                            0.168967  0.948718   0.168967  0.948718   

      var_loss   var_acc  learning_rate  batch_size  num_epochs  filter  \
1236  0.019914  0.008096         0.0001         128          40      64   

      filter2  dropout_rate  layers optimise                 loss  \
1236       16           0.3       2     Adam  binary_crossentropy   

     default_activation  
1236               relu  
R^2 Score: -0.04354901002381384
Mean Squared Error: 0.0015720790523183138

Impact of Hyperparameters on Accuracy (from Linear Regression):
  Hyperparameter   Coefficient
0  learning_rate -4.089893e-02
1     num_epochs  1.256454e-03
2         filter  3.558193e-04
3        filter2 -3.469447e-18
4         layers -1.700406e-02
5   dropout_rate -2.336511e-02
6     batch_size  0.000000e+00

Hyperparameter Importance for Accuracy (from Random Forest)